# 抽卡机

## v2
**思路：**
1. 用 `selenium` 完成授权，并获得抽卡页面的 cookie
2. 用 `requests` 携带之前保存下来的 cookie 发送抽卡请求

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def draw_card_v2(username: '用户名', 
                 password: '密码', 
                 times: '抽卡次数', 
                 action: '1表示抽一次，2表示抽十一次'=1) -> None:
    
    least_wealth = 500 if action == 1 else 5000
    
    # 获得cookies
    chrome_options = Options()
    chrome_options.add_argument("--headless") # 不显示实际界面
    chrome_options.add_argument('--window-size=1920x1080') # 默认600x800，不加的话可能找不到按钮
    driver = webdriver.Chrome(options=chrome_options)

    # 1. 登录CC98登录中心
    driver.get('https://openid.cc98.org/Account/LogOn')
    driver.find_element_by_id('UserName').send_keys(username)
    driver.find_element_by_id('Password').send_keys(password)
    driver.find_element_by_xpath('//button[text()="立即登录"]').click()
    
    # 2. 授权给CC98抽卡中心
    driver.get('https://card.cc98.org/')
    driver.find_element_by_xpath('//*[text()="登录"]').click()
    driver.find_element_by_xpath("//button[@class='btn btn-success']").click()

    driver.get_cookies()
    cookies = {c['name']: c['value'] for c in driver.get_cookies()}

    # 初始化session
    s = requests.Session()
    
    # 获取CSRF验证
    r = s.get('https://card.cc98.org/Draw', cookies=cookies)
    bs = BeautifulSoup(r.text, 'html.parser')
    data = {'__RequestVerificationToken':bs.find('input', {'name':'__RequestVerificationToken'})['value'],
            'X-Requested-With': 'XMLHttpRequest'}
        
    while times > 0:
        current_wealth = requests.get("https://api.cc98.org/user/name/{}".format(username)).json()['wealth']
        if current_wealth < least_wealth:
            print('财富值不足。')
            return
        if action == 1:
            print('当前财富值：{}, 抽一次：'.format(current_wealth), end=' ')
        else:
            print('当前财富值：{}, 抽十一次：'.format(current_wealth), end=' ')
        
        
        # 开抽！
        r = s.post('https://card.cc98.org/Draw/Run/{}'.format(action), 
                   data=data,
                   headers={"content-type":"application/x-www-form-urlencoded; charset=UTF-8"},
                   cookies=cookies)
        bs = BeautifulSoup(r.text, 'html.parser')
    
        N_cards = bs.find_all('div', {'data-level':'0'})
        R_cards = bs.find_all('div', {'data-level':'1'})
        SR_cards = bs.find_all('div', {'data-level':'2'})
        SSR_cards = bs.find_all('div', {'data-level':'3'})
        M_cards = bs.find_all('div', {'data-level':'4'})
        print('抽到{}张N卡，{}张R卡，{}张SR卡，{}张SSR卡，{}张M卡！'.format(len(N_cards), len(R_cards), len(SR_cards), len(SSR_cards), len(M_cards)))
        times -= 1

## v2.1
**思路：**
1. 用 `requestium` 完成授权，并将抽卡页面的 cookie 导入 session
2. 用 session 发送抽卡请求

In [2]:
import requests
from bs4 import BeautifulSoup
from requestium import Session, Keys

def draw_card_v2_1(username: '用户名', 
                 password: '密码', 
                 times: '抽卡次数', 
                 action: '1表示抽一次，2表示抽十一次'=1) -> None:
    
    least_wealth = 500 if action == 1 else 5000
    
    # 初始化session
    s = Session(webdriver_path='/usr/local/bin/chromedriver',
                browser='chrome',
                webdriver_options={'arguments': ['headless', 'window-size=1920x1080']})
    
    # 1. 登录CC98登录中心
    s.driver.get('https://openid.cc98.org/Account/LogOn')
    s.driver.find_element_by_id('UserName').send_keys(username)
    s.driver.find_element_by_id('Password').send_keys(password, Keys.ENTER)
    
    # 2. 授权给CC98抽卡中心
    s.driver.get('https://card.cc98.org/')
    s.driver.find_element_by_xpath('//*[text()="登录"]').click()
    s.driver.find_element_by_xpath("//button[@class='btn btn-success']").click()

    s.transfer_driver_cookies_to_session()

    # 获取CSRF验证
    r = s.get('https://card.cc98.org/Draw')
    bs = BeautifulSoup(r.text, 'html.parser')
    data = {'__RequestVerificationToken':bs.find('input', {'name':'__RequestVerificationToken'})['value'],
            'X-Requested-With': 'XMLHttpRequest'}
        
    while times > 0:
        current_wealth = requests.get("https://api.cc98.org/user/name/{}".format(username)).json()['wealth']
        if current_wealth < least_wealth:
            print('财富值不足。')
            return
        if action == 1:
            print('当前财富值：{}, 抽一次：'.format(current_wealth), end=' ')
        else:
            print('当前财富值：{}, 抽十一次：'.format(current_wealth), end=' ')
        
        
        # 开抽！
        r = s.post('https://card.cc98.org/Draw/Run/{}'.format(action), 
                   data=data,
                   headers={"content-type":"application/x-www-form-urlencoded; charset=UTF-8"})
        bs = BeautifulSoup(r.text, 'html.parser')
    
        N_cards = bs.find_all('div', {'data-level':'0'})
        R_cards = bs.find_all('div', {'data-level':'1'})
        SR_cards = bs.find_all('div', {'data-level':'2'})
        SSR_cards = bs.find_all('div', {'data-level':'3'})
        M_cards = bs.find_all('div', {'data-level':'4'})
        print('抽到{}张N卡，{}张R卡，{}张SR卡，{}张SSR卡，{}张M卡！'.format(len(N_cards), len(R_cards), len(SR_cards), len(SSR_cards), len(M_cards)))
        times -= 1

## v3
**思路：**
1. 只用 `requests` 完成登录、授权、抽卡。

In [3]:
import requests
from bs4 import BeautifulSoup

def draw_card_v3(username: '用户名', 
                 password: '密码', 
                 times: '抽卡次数', 
                 action: '1表示抽一次，2表示抽十一次'=1) -> None:
    
    least_wealth = 500 if action == 1 else 5000
    
    # 1. 登录CC98登录中心
    s = requests.session()
    r = s.get('https://openid.cc98.org/Account/LogOn')
    bs = BeautifulSoup(r.text, 'html.parser')

    data = {
      'UserName': username,
      'Password': password,
      'ValidTime': '',
      '__RequestVerificationToken': bs.find('input', {'name':'__RequestVerificationToken'})['value']
    }

    r = s.post('https://openid.cc98.org/Account/LogOn', 
               headers={'content-type': 'application/x-www-form-urlencoded'}, 
               data=data)

    r = s.get('https://card.cc98.org/Account/LogOn')

    # 2. 授权给CC98抽卡中心
    r = s.get(r.url)
    bs = BeautifulSoup(r.text, 'html.parser')
    data = {
        'Scopes': 'openid',
        'IsConsent': 'true',
    #     'Scopes': 'profile',
        'RememberConsent': 'false', 
        '__RequestVerificationToken': bs.find('input', {'name':'__RequestVerificationToken'})['value']
    }
    r = s.post(r.url, 
               headers={'content-type': 'application/x-www-form-urlencoded'}, 
               data=data)

    bs = BeautifulSoup(r.text, 'html.parser')
    data = {
        'code': bs.find('input', {'name':'code'})['value'],
        'id_token': bs.find('input', {'name':'id_token'})['value'],
        'scope': bs.find('input', {'name':'scope'})['value'],
        'state': bs.find('input', {'name':'state'})['value'],
        'session_state': bs.find('input', {'name':'session_state'})['value'],
    }

    r = s.post('https://card.cc98.org/signin-cc98',
               headers={'content-type': 'application/x-www-form-urlencoded'}, 
               data=data)
    r = s.get('https://card.cc98.org/')

    # 3. 抽卡
    r = s.get('https://card.cc98.org/Draw')
    bs = BeautifulSoup(r.text, 'html.parser')
    data = {'__RequestVerificationToken':bs.find('input', {'name':'__RequestVerificationToken'})['value'],
            'X-Requested-With': 'XMLHttpRequest'}
        
    while times > 0:
        current_wealth = requests.get("https://api.cc98.org/user/name/{}".format(username)).json()['wealth']
        if current_wealth < least_wealth:
            print('财富值不足。')
            return
        if action == 1:
            print('当前财富值：{}, 抽一次：'.format(current_wealth), end=' ')
        else:
            print('当前财富值：{}, 抽十一次：'.format(current_wealth), end=' ')
        
        
        # 开抽！
        r = s.post('https://card.cc98.org/Draw/Run/{}'.format(action), 
                   data=data,
                   headers={"content-type":"application/x-www-form-urlencoded; charset=UTF-8"},
                  )
        bs = BeautifulSoup(r.text, 'html.parser')
    
        N_cards = bs.find_all('div', {'data-level':'0'})
        R_cards = bs.find_all('div', {'data-level':'1'})
        SR_cards = bs.find_all('div', {'data-level':'2'})
        SSR_cards = bs.find_all('div', {'data-level':'3'})
        M_cards = bs.find_all('div', {'data-level':'4'})
        print('抽到{}张N卡，{}张R卡，{}张SR卡，{}张SSR卡，{}张M卡！'.format(len(N_cards), len(R_cards), len(SR_cards), len(SSR_cards), len(M_cards)))

        times -= 1

## v4
**思路：**
1. 在 v3 的基础上增加了「查看抽卡情况总览」和「查看欧皇指数」功能

**注：**欧皇指数中用到的 `weights` 为每种卡对应概率的倒数

| 等级 | N      | R      | SR     | SSR   | Mystery |
| ---- | ------ :| ------ :| ------ :| ----- | ------- :|
| 比例 | 53.49% | 30.00% | 15.00% | 1.50% | 0.01%   |
| 权重 | 1.87| 3.33 | 6.67 | 66.67 | 10000 |

In [4]:
import requests
from bs4 import BeautifulSoup
from prettytable import PrettyTable
from pyecharts.charts import Gauge

def draw_card_v4(username: '用户名', 
                 password: '密码', 
                 times: '抽卡次数', 
                 action: '1表示抽一次，2表示抽十一次'=1) -> None:
    
    least_wealth = 500 if action == 1 else 5000
    
    # 1. 登录CC98登录中心
    s = requests.session()
    r = s.get('https://openid.cc98.org/Account/LogOn')
    bs = BeautifulSoup(r.text, 'html.parser')

    data = {
      'UserName': username,
      'Password': password,
      'ValidTime': '',
      '__RequestVerificationToken': bs.find('input', {'name':'__RequestVerificationToken'})['value']
    }

    r = s.post('https://openid.cc98.org/Account/LogOn', 
               headers={'content-type': 'application/x-www-form-urlencoded'}, 
               data=data)

    r = s.get('https://card.cc98.org/Account/LogOn')

    # 2. 授权给CC98抽卡中心
    r = s.get(r.url)
    bs = BeautifulSoup(r.text, 'html.parser')
    data = {
        'Scopes': 'openid',
        'IsConsent': 'true',
        'RememberConsent': 'false', 
        '__RequestVerificationToken': bs.find('input', {'name':'__RequestVerificationToken'})['value']
    }
    r = s.post(r.url, 
               headers={'content-type': 'application/x-www-form-urlencoded'}, 
               data=data)

    bs = BeautifulSoup(r.text, 'html.parser')
    data = {
        'code': bs.find('input', {'name':'code'})['value'],
        'id_token': bs.find('input', {'name':'id_token'})['value'],
        'scope': bs.find('input', {'name':'scope'})['value'],
        'state': bs.find('input', {'name':'state'})['value'],
        'session_state': bs.find('input', {'name':'session_state'})['value'],
    }
    r = s.post('https://card.cc98.org/signin-cc98',
               headers={'content-type': 'application/x-www-form-urlencoded'}, 
               data=data)
    r = s.get('https://card.cc98.org/')

    # 3. 抽卡
    r = s.get('https://card.cc98.org/Draw')
    bs = BeautifulSoup(r.text, 'html.parser')
    data = {'__RequestVerificationToken':bs.find('input', {'name':'__RequestVerificationToken'})['value'],
            'X-Requested-With': 'XMLHttpRequest'}
        
    count = num_N = num_R = num_SR = num_SSR = num_M = 0
    while count < times:
        current_wealth = requests.get("https://api.cc98.org/user/name/{}".format(username)).json()['wealth']
        if current_wealth < least_wealth:
            print('财富值不足。')
            break
        if action == 1:
            print('当前财富值：{}, 抽一次：'.format(current_wealth), end=' ')
        else:
            print('当前财富值：{}, 抽十一次：'.format(current_wealth), end=' ')
        
        
        # 开抽！
        r = s.post('https://card.cc98.org/Draw/Run/{}'.format(action), 
                   data=data,
                   headers={"content-type":"application/x-www-form-urlencoded; charset=UTF-8"},
                  )
        bs = BeautifulSoup(r.text, 'html.parser')
    
        N_cards = bs.find_all('div', {'data-level':'0'})
        R_cards = bs.find_all('div', {'data-level':'1'})
        SR_cards = bs.find_all('div', {'data-level':'2'})
        SSR_cards = bs.find_all('div', {'data-level':'3'})
        M_cards = bs.find_all('div', {'data-level':'4'})
        
        num_N += len(N_cards)
        num_R += len(R_cards)
        num_SR += len(SR_cards)
        num_SSR += len(SSR_cards)
        num_M += len(M_cards)
        count += 1
        print('抽到{}张N卡，{}张R卡，{}张SR卡，{}张SSR卡，{}张M卡！'.format(len(N_cards), len(R_cards), len(SR_cards), len(SSR_cards), len(M_cards)))
    
    num_cards = [num_N, num_R, num_SR, num_SSR, num_M]
    table = PrettyTable(['\033[33m等级\033[0m','N', 'R', 'SR', 'SSR', 'Mystery'])
    table.add_row(['\033[33m数量\033[0m'] + num_cards)
    print('\n\033[1;31m抽卡情况总览:\033[0m\n', table)
    
    weights = [1.87, 3.33, 6.67, 66.67, 10000]
    index = min(int(10 * sum([i * j for i, j in zip(weights, num_cards)]) / sum(num_cards)), 100)
    return Gauge().add("", [('欧皇指数', index)])

In [ ]:
%%time
draw_card_v2(username='', password='', times=1, action=2)

In [ ]:
%%time
draw_card_v2_1(username='', password='', times=1, action=2)

In [ ]:
%%time
draw_card_v3(username='', password='', times=1, action=2)

In [ ]:
%%time
gauge = draw_card_v4(username='', password='', times=1, action=2)
gauge.render_notebook()